In [3]:
import numpy as np
import pandas as pd
import itertools

# dataPath =Data/carSales.csvs.csv"
dataPath ="Data/carSales1.csv"

'''
读取数据
str_col:  可以用作subspace的列
num_col:  measure的列
'''

df = pd.read_csv(dataPath)
df = df.fillna(0)
num_col = df.select_dtypes(include=np.number).columns.tolist()
str_col = [str for str in df.columns.tolist() if str not in num_col]

print("String Column: ", str_col)
print("Numeric Column: ", num_col)

String Column:  ['Year', 'Brand', 'Category']
Numeric Column:  ['Sales']


## Extract Top 1 Insight using QuickInsight Format

In [4]:
'''
获取每一个subspace的range
'''
dim_range = dict()
for dim in str_col:
    dim_range[dim] = []
    dim_range[dim].extend(df[dim].unique())
print(dim_range)

{'Year': ['2007/1/1', '2008/1/1', '2009/1/1', '2010/1/1', '2011/1/1'], 'Brand': ['BMW', 'Ford', 'GMC', 'Honda', 'Hyundai', 'Mazda', 'Toyota', 'Volkswagen'], 'Category': ['Compact', 'Fullsize', 'Midsize', 'Sporty', 'Subcompact', 'SUV', 'Pickup', 'MPV']}


In [8]:
def format_subspace(subspace):
    '''
    subspace输出格式化
    '''
    subspace_string = []
    for dim in str_col:
        exist = False
        for space in subspace:
            if space in dim_range[dim]:
                exist = True
                subspace_string.append(space)
        if not exist:
            subspace_string.append("*")
    return subspace_string

In [9]:
for subspaceLen in range(0, len(str_col)):
#     subspaceLen: subspace的个数
    for dims in itertools.combinations(str_col, subspaceLen):
#       dims: subspace中的元素
        breakdowns = [x for x in str_col if (x not in dims)]
        space_range = []
        for dim in dims:
            space_range.append(dim_range[dim])
        combi = list(itertools.product(*space_range))
        for subspace in combi:
            for measure in num_col:
                if (len(dims)>0):
                    dff = df.loc[np.all([df[dims[i]] == subspace[i] for i in range(0, subspaceLen)], axis=0)]
                else:
                    dff =df
#               df: subspace取特定值后得到的dataframe
                for bd in breakdowns:
                    df_group = dff.groupby(bd, as_index=False).mean()
#                   现在groupby用的是平均值

                    if not df_group.empty:
                        idx = df_group[measure].idxmax()
#                       获得Top1的index
                        res_str = ';'.join(format_subspace(subspace))
                        print('{},{},{},{},{}'.format(res_str, bd, measure, df_group.at[idx, measure], df_group.at[idx, bd]))

*;*;*,Year,Sales,91726.56363636363,2007/1/1
*;*;*,Brand,Sales,177257.04,Honda
*;*;*,Category,Sales,149148.97435897434,Midsize
2007/1/1;*;*,Brand,Sales,206827.2,Honda
2007/1/1;*;*,Category,Sales,179617.42857142858,Midsize
2008/1/1;*;*,Brand,Sales,203119.6,Honda
2008/1/1;*;*,Category,Sales,150622.0,Midsize
2009/1/1;*;*,Brand,Sales,164125.4,Honda
2009/1/1;*;*,Category,Sales,131875.0,Midsize
2010/1/1;*;*,Brand,Sales,163012.6,Honda
2010/1/1;*;*,Category,Sales,142726.25,Midsize
2011/1/1;*;*,Brand,Sales,149200.4,Honda
2011/1/1;*;*,Category,Sales,144712.75,Midsize
*;BMW;*,Year,Sales,34151.22222222222,2007/1/1
*;BMW;*,Category,Sales,99222.83333333333,Compact
*;Ford;*,Year,Sales,117825.18181818182,2011/1/1
*;Ford;*,Category,Sales,189015.6,Midsize
*;GMC;*,Year,Sales,70776.66666666667,2007/1/1
*;GMC;*,Category,Sales,83114.4,Pickup
*;Honda;*,Year,Sales,206827.2,2007/1/1
*;Honda;*,Category,Sales,314133.4,Midsize
*;Hyundai;*,Year,Sales,117909.2,2011/1/1
*;Hyundai;*,Category,Sales,161107.4,Midsize
*;M